In [1]:
# Instalar las librerias
# !pip install pandas
# !pip install scikit-learn
# D:\01_MaestriaAnalitica_UnivAndes\07_ML_NLP\S4\Proyecto\pry_pric_cars\venv\Scripts\pip.exe install -r requirements.txt
# !pip install imblearn

In [2]:
import numpy as np
import xgboost as xgb
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.utils import class_weight
from sklearn import preprocessing
import time

import pandas as pd
import os

# Modelado
# ==============================================================================
from sklearn.neural_network import MLPRegressor
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold
from sklearn import set_config
import multiprocessing
from joblib import dump, load

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')


In [3]:
directorio = os.path.split(os.getcwd())[0]
test_df = pd.read_csv(os.path.join("../INPUT/","dataTest_carListings.csv")).replace(' ', '_', regex=True)
train_df = pd.read_csv(os.path.join("../INPUT/","dataTrain_carListings.csv")).replace(' ', '_', regex=True)

X_Train = train_df.iloc[:,1:]
X_Test = test_df.iloc[:,1:]
y_Train = train_df.iloc[:,0]
y_Test = test_df.iloc[:,0]

X_Train

,Year,Mileage,State,Make,Model
0,2017,9913,_FL,Jeep,Wrangler
1,2015,20578,_OH,Chevrolet,Tahoe4WD
2,2012,83716,_TX,BMW,X5AWD
3,2014,28729,_OH,Cadillac,SRXLuxury
4,2013,64032,_CO,Jeep,Wrangler
...,...,...,...,...,...
399995,2015,25287,_TX,Lexus,RXRX
399996,2015,17677,_MI,Chevrolet,EquinoxFWD
399997,2014,66688,_NC,Buick,EnclaveLeather
399998,2014,37872,_IN,Volkswagen,Jetta


# Preprocesamiento

## Reescalado

In [4]:
categoricas = ['State','Make','Model']
numericas = ['Year','Mileage']

X_var_numericas = X_Train[numericas]
X_std_scaller = preprocessing.StandardScaler()

X_Train_StdSca = X_std_scaller.fit_transform(X_var_numericas)
X_Test_StdSca = X_std_scaller.fit_transform(X_Test[numericas])

dump(X_std_scaller, '../OUTPUT/std_scaler.bin', compress=True)
#sc=load('std_scaler.bin')

## Estandarizar categoricas

X_var_categoricas = X_Train[categoricas]
enc = preprocessing.OneHotEncoder(sparse=False)
enc.fit(X_Train[categoricas]) # en la particion de los conjuntos de entrenamiento se pierden categorias, por lo tanto es necesario hacerlo con toda la data

X_train_OneHot = enc.transform(X_Train[categoricas])
X_test_OneHot = enc.transform(X_Test[categoricas])

dump(enc, '../OUTPUT/one_hot_scaller.bin', compress=True)

## Matrices X de entrenamiento y prueba
X_Train_std = np.hstack([X_Train_StdSca,X_train_OneHot])
X_Test_std = np.hstack([X_Train_StdSca,X_train_OneHot])





# Entrenamiento del modelo


In [5]:
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

xgb = XGBRegressor(tree_method = "gpu_hist",max_features="sqrt")
xgb.fit(X_Train_std, y_Train)
preds_xgboost = xgb.predict(X_Test_std)

# Cuando se seleccione el mejor modelo, se almacena
dump(xgb, '../OUTPUT/best_model_price_cars.bin', compress=True)

[19:00:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/learner.cc:627: 
Parameters: { "max_features" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




['../OUTPUT/best_model_price_cars.bin']

In [6]:
' FL'.replace(" ","")



'FL'

In [7]:
# predict/?PRECIO=1500&ESTADO=FL&MILEAGE=2000
entrada = {
    "Year":2014,
    "Mileage":31909,
    "State":' FL',
    "Make":'Nissan', 
    "Model": 'MuranoAWD'    
}



In [8]:
X_Train

,Year,Mileage,State,Make,Model
0,2017,9913,_FL,Jeep,Wrangler
1,2015,20578,_OH,Chevrolet,Tahoe4WD
2,2012,83716,_TX,BMW,X5AWD
3,2014,28729,_OH,Cadillac,SRXLuxury
4,2013,64032,_CO,Jeep,Wrangler
...,...,...,...,...,...
399995,2015,25287,_TX,Lexus,RXRX
399996,2015,17677,_MI,Chevrolet,EquinoxFWD
399997,2014,66688,_NC,Buick,EnclaveLeather
399998,2014,37872,_IN,Volkswagen,Jetta


In [9]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import train_test_split
# import xgboost as xgb
# import time

# # Train the model without parallelization
# params = {'objective': 'multi:softmax', 'num_class': 7, 'n_jobs': 1}

# print("Training model without parallelization.")
# start_time = time.time()
# model = xgb.train(params, xgb.DMatrix(X_train, label=y_train))
# elapsed_time_no_parallel = time.time() - start_time

# # Train the model with parallelization
# params_parallel = {'objective': 'multi:softmax', 'num_class': 7, 'n_jobs': -1}

# print("Training model with parallelization.")
# start_time_parallel = time.time()
# model_parallel = xgb.train(params_parallel, xgb.DMatrix(X_train, label=y_train))
# elapsed_time_parallel = time.time() - start_time_parallel

# # Print the time comparisons
# print(f"Training time without parallelization: {elapsed_time_no_parallel:.2f} seconds")
# print(f"Training time with parallelization: {elapsed_time_parallel:.2f} seconds")
